# Integrators and sampling

In [1]:
# Preliminary imports
from simtk import openmm, unit
from simtk.openmm import app
import numpy as np

To explore the integrators available for sampling in OpenMM, we'll use the alanine dipeptide in vacuum test system imported from [`openmmtools`](http://openmmtools.readthedocs.io):

In [2]:
# Load in an AMBER system
from simtk.openmm import app
pdb = app.PDBFile('resources/alanine-dipeptide-implicit.pdb')
prmtop = app.AmberPrmtopFile('resources/alanine-dipeptide-implicit.prmtop')
inpcrd = app.AmberInpcrdFile('resources/alanine-dipeptide-implicit.inpcrd')
system = prmtop.createSystem(nonbondedMethod=app.NoCutoff, implicitSolvent=app.OBC2, constraints=app.HBonds)
positions = inpcrd.getPositions(asNumpy=True)
topology = pdb.getTopology()

# Compute the potential energy
def compute_potential(system, positions):
    """Print the potential energy given a System and positions."""
    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator)
    context.setPositions(positions)
    print('Potential energy: %s' % context.getState(getEnergy=True).getPotentialEnergy())
    # Clean up
    del context, integrator
    
compute_potential(system, positions)

Potential energy: -137.4394352560201 kJ/mol


## Integrators available for use

There are several ways to specify which integrators to use for sampling:
* Use one of the [built-in OpenMM integrators](http://docs.openmm.org/7.1.0/api-python/library.html#integrators)
* Use the powerful [`CustomIntegrator`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.CustomIntegrator.html#simtk.openmm.openmm.CustomIntegrator) facility to define your own integrator in terms of basic algebraic operations on positions and velocities
* Use one of the high-quality integrators from [`openmmtools.integrators`](http://openmmtools.readthedocs.io/en/latest/integrators.html)
* Write your own integrator plugin using C++ (and CUDA or OpenCL)

## Built-in OpenMM integrators

OpenMM provides a number of built-in [`Integrators`](http://docs.openmm.org/7.1.0/userguide/application.html#integrators) that may be useful for your intended application, but also provides a very flexible way to efficiently define new integrators that can still be executed very efficiently on the GPU.

All OpenMM integrators are derived from the [`Integrator`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Integrator.html#simtk.openmm.openmm.Integrator) base class that provides some baseline functionality such that all integrators work the same way. 

OpenMM provides a [large number of integrators you can use directly](http://docs.openmm.org/7.1.0/api-python/library.html#integrators). Here are just a few of them:
* [`VerletIntegrator`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.VerletIntegrator.html#simtk.openmm.openmm.VerletIntegrator): A leapfrog Verlet integrator where the positions and velocities are half a timestep out of sync. This is useful in many situations, but care must be taken regarding the half timestep lag.
* [`LangevinIntegrator`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.LangevinIntegrator.html#simtk.openmm.openmm.LangevinIntegrator): A [leapfrog Langevin](https://github.com/pandegroup/openmm/issues/1766#issuecomment-289615072) / [VVVR](http://dx.doi.org/10.1103/PhysRevX.3.011007) / [Bussi-Parrinello](10.1103/PhysRevE.75.056707) Langevin integrator where the velocity is shifted from the positions by half a timestep.
* [`MTSIntegrator`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.mtsintegrator.MTSIntegrator.html#simtk.openmm.mtsintegrator.MTSIntegrator): A multiple-timestep integrator following [rRESPA from Tuckerman and Berne](http://dx.doi.org/10.1063/1.463137).
* [`AMDIntegrator`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.amd.AMDIntegrator.html#simtk.openmm.amd.AMDIntegrator): An integrator that implements the [accelerated MD algorithm of McCammon](https://doi.org/10.1063/1.2789432), though care must be taken in noting the potential energy reported is not actually modified.
* Experimental variable-timestep integrators [`VariableVerletIntegrator`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.VariableVerletIntegrator.html#simtk.openmm.openmm.VariableVerletIntegrator) and [`VariableLangevinIntegrator`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.VariableLangevinIntegrator.html#simtk.openmm.openmm.VariableLangevinIntegrator) that should be used cautiously since they are no longer sympletic, but may be useful when forces are very large.

### OpenMM's `LangevinIntegrator`

Let's create a [`LangevinIntegrator`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.LangevinIntegrator.html#simtk.openmm.openmm.LangevinIntegrator), which uses a version of [velocity Verlet with velocity randomization (VVVR)](http://dx.doi.org/10.1103/PhysRevX.3.011007), also known as the [Bussi-Parrinello Langevin integrator](10.1103/PhysRevE.75.056707), where the velocity is shifted from the positions by half a timestep.

We first have to create a [`LangevinIntegrator`](http://docs.openmm.org/7.1.0/api-python/library.html#integrators) and bind it to a [`Context`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Context.html#simtk.openmm.openmm.Context). 

In [3]:
# Create a new integrator since the previously-created integrator was irrevocably bound to the previous Context
temperature = 298.0 * unit.kelvin
collision_rate = 91.0 / unit.picosecond
timestep = 2.0 * unit.femtoseconds
integrator = openmm.LangevinIntegrator(temperature, collision_rate, timestep)
# Create a Context for this integrator
context = openmm.Context(system, integrator)
# Set the positions
context.setPositions(positions)
# Minimize the potential energy
openmm.LocalEnergyMinimizer.minimize(context)

The initial velocities are, by default, zero, but we can select initial velocities from the Maxwell distribution with [`context.setVelocitiesToTemperature()`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Context.html#simtk.openmm.openmm.Context.setVelocitiesToTemperature):

In [4]:
# Set velocities from Maxwell-Boltzmann distribution
context.setVelocitiesToTemperature(temperature)

To integrate a trajectory, we call [`integrator.step`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Integrator.html#simtk.openmm.openmm.Integrator.step):

In [5]:
# Integrate some dynamics
nsteps = 100 # number of integrator steps
integrator.step(nsteps)

We can write a little loop to report some information every few timesteps:

In [6]:
# Run a few iterations of a few steps each, reporting potential energy
for iteration in range(10):
    integrator.step(100)
    state = context.getState(getEnergy=True)
    print('%8.3f ps : potential %12.6f kJ/mol' % (state.getTime() / unit.picoseconds, state.getPotentialEnergy() / unit.kilojoules_per_mole))

   0.400 ps : potential  -131.806771 kJ/mol
   0.600 ps : potential  -111.863356 kJ/mol
   0.800 ps : potential   -92.643765 kJ/mol
   1.000 ps : potential  -123.776344 kJ/mol
   1.200 ps : potential  -117.666694 kJ/mol
   1.400 ps : potential  -122.445981 kJ/mol
   1.600 ps : potential  -121.708050 kJ/mol
   1.800 ps : potential  -110.365573 kJ/mol
   2.000 ps : potential   -94.195386 kJ/mol
   2.200 ps : potential   -96.800356 kJ/mol


## The `Simulation` convenience class

While we could write our own wrapper to run a simulation and write data to disk, OpenMM's [`app` layer](http://docs.openmm.org/7.1.0/api-python/app.html) provides the [`Simulation` class](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.app.simulation.Simulation.html#simtk.openmm.app.simulation.Simulation) to help you do this using a modular Python-based plugin architecture to specify which and how data should be stored. The [user guide](http://docs.openmm.org/7.1.0/userguide/application.html#a-first-example) provides a nice overview of this.

### Running a `Simulation` that writes data to the terminal

For example, to run a simulation that prints data to the terminal, we can use:

In [7]:
from sys import stdout
integrator = openmm.LangevinIntegrator(temperature, collision_rate, timestep)
simulation = app.Simulation(topology, system, integrator)
simulation.context.setPositions(positions)
simulation.minimizeEnergy()
simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, potentialEnergy=True, temperature=True))
simulation.step(1000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,-114.42471549690138,221.37050469713836
200,-103.20719201083,254.81766233549368
300,-118.09064566365304,309.0930921889871
400,-93.2987613880557,237.69745442208156
500,-117.24882573340804,288.80034436587334
600,-97.29203455519932,365.6957345976407
700,-105.15449187321819,302.16964000449804
800,-111.00359316523141,332.42557584314795
900,-95.51387774166336,273.07480002136003
1000,-117.1708525191791,233.73798637207642


### Running a `Simulation` that writes a trajectory to disk

For example, to run a simulation that writes data in the extensible MDTraj HDF5 format, Amber NetCDF format, or CHARMM DCD format, we can use the [MDTraj Reporters](http://mdtraj.org/1.6.2/api/reporters.html):

In [8]:
import mdtraj
integrator = openmm.LangevinIntegrator(temperature, collision_rate, timestep)
simulation = app.Simulation(topology, system, integrator)
simulation.context.setPositions(positions)
simulation.minimizeEnergy()
reportInterval = 10
simulation.reporters.append(mdtraj.reporters.HDF5Reporter('output.h5', reportInterval, coordinates=True, time=True, cell=True, potentialEnergy=True, temperature=True))
simulation.reporters.append(mdtraj.reporters.DCDReporter('output.dcd', reportInterval))
simulation.reporters.append(mdtraj.reporters.NetCDFReporter('output.nc', reportInterval))
simulation.step(2000)
del simulation # Make sure to close all files

/opt/conda/lib/python3.7/site-packages/mdtraj/utils/validation.py:116: TypeCastPerformanceWarning: Casting xyz dtype=float64 to <class 'numpy.float32'> 
  TypeCastPerformanceWarning)


We can easily view a trajectory in MDTraj.

In [9]:
traj = mdtraj.load('output.h5', 'r')
import nglview
view = nglview.show_mdtraj(traj)
view.add_ball_and_stick('all')
view.center(zoom=True)
view

_ColormakerRegistry()

NGLWidget(max_frame=199)